importing the required libraries

In [1]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aj240\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df=pd.read_csv("tennis_articles - tennis_articles.csv")

In [3]:
df.head()

,article_id,article_title,article_text,source
0,1,"I do not have friends in tennis, says Maria Sh...",Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,Federer defeats Medvedev to advance to 14th Sw...,"BASEL, Switzerland (AP) — Roger Federer advanc...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Tennis: Roger Federer ignored deadline set by ...,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Nishikori to face off against Anderson in Vien...,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,Roger Federer has made this huge change to ten...,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


In [4]:
print(df["source"][0])

https://www.tennisworldusa.org/tennis/news/Maria_Sharapova/62220/i-do-not-have-friends-in-tennis-says-maria-sharapova/


In [5]:
print(df['article_text'][0])

Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net. So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in the same 

importing the required natural language processing tools

In [6]:
from nltk.tokenize import sent_tokenize

extracting sentences from the article text

In [7]:
sentences=[]
for s in df["article_text"]:
    sentences.append(sent_tokenize(s))

In [8]:
sentences

[['Maria Sharapova has basically no friends as tennis players on the WTA Tour.',
  "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.",
  'I think everyone knows this is my job here.',
  "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.",
  "So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.",
  "I'm a pretty competitive girl.",
  "I say my hellos, but I'm not sending any players flowers as well.",
  "Uhm, I'm not really friendly or close to many players.",
  "I have not a lot of friends away from the courts.'",
  'When she said she is not really close to a lot of players, is that something strategic that she is doing?',
  "Is it different on the men's tour than the women's tour?",
  

flattening the list

In [9]:
sentences=[y for x in sentences for y in x]

In [10]:
print(sentences[:5])

['Maria Sharapova has basically no friends as tennis players on the WTA Tour.', "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.", 'I think everyone knows this is my job here.', "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.", "So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match."]


we use GloVe word embeddings to create vector representation of our sentences

In [12]:
pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9686 sha256=3a2e971c692d6a4edbcdc8a2f3ff79c65c1fbd53100a8816195a035419236d0f
  Stored in directory: c:\users\aj240\appdata\local\pip\cache\wheels\bd\a8\c3\3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
Note: you may need to restart the kernel to use updated packages.


In [15]:
import wget
url = 'http://nlp.stanford.edu/data/glove.6B.zip'
filename = wget.download(url)

100% [......................................................................] 862182613 / 862182613

In [16]:
# importing required modules 
from zipfile import ZipFile 

# opening the zip file in READ mode 
with ZipFile(filename, 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 

File Name                                             Modified             Size
glove.6B.50d.txt                               2014-08-04 13:15:00    171350079
glove.6B.100d.txt                              2014-08-04 13:14:34    347116733
glove.6B.200d.txt                              2014-08-04 13:14:44    693432828
glove.6B.300d.txt                              2014-08-27 12:19:16   1037962819
Extracting all the files now...
Done!


ectracting the word embeddings(vectors)

In [17]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [18]:
len(word_embeddings)

400000

# TEXT PROCESSING

In [19]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [43]:
#removing stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aj240\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
clean_sentences[:5]

['maria sharapova basically friends tennis players wta tour',
 'russian player problems openly speaking recent interview said really hide feelings much',
 'think everyone knows job',
 'courts court playing competitor want beat every single person whether locker room across net',
 'one strike conversation weather know next minutes go try win tennis match']

# Representing sentences as vectors

In [23]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [29]:
print(sentence_vectors[0])
len(sentence_vectors[0])

[ 5.14825583e-02  1.10544682e-01  6.94999397e-01  1.89168096e-01
 -9.58077684e-02  3.20288986e-01  2.70662010e-01  5.42440832e-01
 -3.05938005e-01 -1.56364068e-01  3.70127618e-01  8.09492469e-02
  8.41393881e-03  2.47571543e-01 -3.69342804e-01 -7.61044994e-02
  8.08582604e-02  2.30643645e-01 -2.70402402e-01  5.13828397e-01
 -6.12548441e-02  3.87900352e-01  1.03121363e-01  7.72494674e-01
  2.59960234e-01 -7.96069205e-02  1.42143592e-01 -9.62644577e-01
  7.54904330e-01  6.03260659e-02 -4.58570123e-01  2.36780301e-01
  2.29152635e-01 -1.56453326e-01  3.97632688e-01 -2.32720934e-02
 -5.05520999e-01  4.13252831e-01 -2.85759270e-01 -1.35231465e-01
 -1.37098104e-01 -1.48972601e-01  3.37537557e-01 -3.49540442e-01
  1.53484434e-01 -2.33341649e-01 -1.98460802e-01 -1.27821520e-01
  5.08063912e-01 -3.68636876e-01 -2.28472307e-01 -3.15306723e-01
  1.36149466e-01  2.22507194e-01  1.19500056e-01 -1.71007359e+00
 -1.04403630e-01  3.45346779e-01  5.54419458e-01  7.91236103e-01
 -2.63593912e-01  5.01183

100

explanation:
first we fetch vectors(each of size 100 elements) for the constituent words in a sentence, we then take mean/average of those vectors to arrive at a consolidated vector for the sentence

so now essentially every sentence is represented by a vector of 100 elements which we calculated by taking the 100 element vectors for each word, adding them all up and the getting the average by dividing it by the number of words in sentence

# Preparation of similarity matrix

now we try to find similarities in the sentences by using the cosine similarity approach.

we first create an empty similarity matrix and populate it with cosine similarities of the sentences.

( Cosine similarity is a measure of similarity between two non-zero vectors
  it measures the cosine of the angle between two vectors projected in a    
  multi-dimensional space)
https://www.machinelearningplus.com/nlp/cosine-similarity/

In [31]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

now we calculate cosine similarity of the sentences and initialize the corresponding position in the similarity matrix with the cosine similarity value

In [32]:
from sklearn.metrics.pairwise import cosine_similarity

for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

now we convert the similarity matrix into a graph(The datatype)
here the nodes will represent the sentences and the edges will represent the similarity scores

In [35]:
# importing networkx  
import networkx as nx 
 
nx_graph = nx.from_numpy_array(sim_mat)

#applying the page rank alg
scores = nx.pagerank(nx_graph)


# summary extraction

now we get the ranked sentences list by sorting the scores for the sentences in descending order

In [36]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [40]:
sentences

['Maria Sharapova has basically no friends as tennis players on the WTA Tour.',
 "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.",
 'I think everyone knows this is my job here.',
 "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.",
 "So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.",
 "I'm a pretty competitive girl.",
 "I say my hellos, but I'm not sending any players flowers as well.",
 "Uhm, I'm not really friendly or close to many players.",
 "I have not a lot of friends away from the courts.'",
 'When she said she is not really close to a lot of players, is that something strategic that she is doing?',
 "Is it different on the men's tour than the women's tour?",
 "'No, not at

In [42]:
# Extract top 10 sentences as the summary
for i in range(10):
  print(ranked_sentences[i][1],"\n")

“I was on a nice trajectorythen,” Reid recalled.“If I hadn’t got sick, I think I could have started pushing towards the second week at the slams and then who knows.” Duringa comeback attempt some five years later, Reid added Bernard Tomic and 2018 US Open Federer slayer John Millman to his list of career scalps. 

Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest. 

So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. 

Speaking at the Swiss Indoors tournament where he will play in Sunday’s final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment. 

Currently in ninth place, Nishikori with a win could move to within 125 points of the cut for the eight-man event in London next mont